In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

In [2]:
confirmed=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

In [3]:
confirmed_country = confirmed.iloc[:, 1:].groupby(['Country/Region']).sum().iloc[:, 2:].reset_index()
deaths_country = deaths.iloc[:, 1:].groupby(['Country/Region']).sum().iloc[:, 2:].reset_index()
recovered_country = recovered.iloc[:, 1:].groupby(['Country/Region']).sum().iloc[:, 2:].reset_index()

In [4]:
confirmed_country=confirmed_country.append(confirmed_country.iloc[:,1:].sum(axis=0), ignore_index=True)
deaths_country=deaths_country.append(deaths_country.iloc[:,1:].sum(axis=0), ignore_index=True)
recovered_country=recovered_country.append(recovered_country.iloc[:,1:].sum(axis=0), ignore_index=True)

In [5]:
countries=confirmed_country.iloc[:, 0].values
dates=confirmed_country.columns.values[1:]

In [7]:
daily_confirmed_country=confirmed_country
for i in range(len(dates)-1, 0, -1):
    daily_confirmed_country[dates[i]]=daily_confirmed_country[dates[i]]-daily_confirmed_country[dates[i-1]]

In [9]:
daily_confirmed_country

NameError: name 'daily_confirmed_country' is not defined